In [220]:
import pandas as pd

%matplotlib inline 
#Load hospital data
merged_data = pd.read_pickle("data/merged_data_7May2020.pkl")

In [221]:
print(merged_data['hospital'].unique())

['AssafHarofe' 'BneiZion' 'Barzilai' 'HilelYaffe' 'Galil' 'Volfson' 'Ziv'
 'Poriya' 'Rambam' 'Shiba']


In [222]:
m = merged_data
m.rename(columns = lambda x : re.sub('\s*:\s*', '_', re.sub('-','_', x)), inplace=True)
m.columns

Index(['hospital', 'Date', 'keep', 'hospitalization_0_1',
       'hospitalization_1_4', 'hospitalization_5_18', 'hospitalization_19_34',
       'hospitalization_35_64', 'hospitalization_65_74',
       'hospitalization_75_84', 'hospitalization_85plus',
       'all_hospitalizations', 'release_0_1', 'release_1_4', 'release_5_18',
       'release_19_34', 'release_35_64', 'release_65_74', 'release_75_84',
       'release_85plus', 'all_releases', 'all_visits'],
      dtype='object')

In [223]:
#Run a smoothing filter on the hospitalization data
from scipy import signal
b, a = signal.butter(14, 0.07)

if (1):
  smoothed_merged_data = merged_data

  for column in smoothed_merged_data:
    if (column != "Date") and (column != "hospital"):
      smoothed_merged_data[column] = signal.filtfilt(b, a, smoothed_merged_data[column], padlen=7)
      #smoothed_merged_data[column] = smoothed_merged_data[column].rolling(window=14).mean()

  merged_data = smoothed_merged_data

In [250]:
air_quality = pd.read_excel("data/FormattedStationData.xlsx")

# there's a stray SO2 sensor for Kiryat Haim for some reason, remove it
air_quality = air_quality.drop('Kiryat Haim : SO2', axis='columns').rename(columns = {'Kiryat Haim : SO2.1' : 'Kiryat Haim : SO2'})

air_quality["DateTime"] = pd.to_datetime(air_quality["Date-Time"],errors='coerce')
air_quality["PollutionDate"] = air_quality["DateTime"].dt.date

#Check if all dates have been processed (should be zero)
print(len(air_quality["DateTime"]) - air_quality["DateTime"].count())

#print(air_quality)

0


In [301]:
# Read air quality sensor data
air_quality = pd.read_excel("data/StationData.xlsx")

# Many columns are empty or sparse. Require 80% availability of data per column
air_quality.dropna(thresh=len(air_quality)*0.8, axis=1, inplace=True)

In [302]:
# Mark column be type of pollutants they measure

import re

air_quality.rename(columns=lambda x: re.sub('[- ()+]','_', re.sub('2\.5', '25', re.sub('_', '__', x))), inplace=True)

colre_gaseous = re.compile('(O3|NO2|NO|NOX|SO2|TOC_SCRUBBER|TOC_RTO|CO|BENZN|O_Xyle|TOLUEN|EthylB|M_P_XY|1_3butadiene|TOL|NOx|H2S|CO TRAFIC|NO2_TRAFIC|NO_TRAFIC|NOX_TRAFIC)$')

colre_particulate = re.compile('(PM10|PM25|pm\(10-2\.5\)|pm_10_25_|DUST|PM1|BLACK CARBON)$')

colre_climate = re.compile('(StWd|RH|WDS|WDD|SR|PREC)$')  # not used, we take climate data from another source
colre_operational = re.compile('(ITemp|TEMP|Filter|LXpk_max|LAF_max|LAF_min|LAim|LXeq|LAeq|FILTER_2.5|spare|PM1_Flow|PM25_Filter|PM10_Noise|PM25_Noise|PM10_Filter|Flow)$')  # not used, this should not matter for anythingg

# There are two data columns which didn't seem to belong anywhere: BP and BRP. Anyone knows what they are?2

all_cols = air_quality.columns

aircols_gaseous = list(filter(colre_gaseous.search, all_cols))
aircols_particulate = list(filter(colre_particulate.search, all_cols))

print(aircols_gaseous)
print(aircols_particulate)


['Shprintzak__O3', 'Shprintzak__NO2', 'Shprintzak__NO', 'Shprintzak__NOX', 'Romema__CO', 'Romema__NO2', 'Romema__NO', 'Romema__NOX', 'Kiryat_Yam__NOX', 'Kiryat_Yam__NO', 'Kiryat_Yam__NO2', 'Kiryat_Yam__O3', 'Kiryat_Tivon__NO2', 'Kiryat_Tivon__O3', 'Kiryat_Tivon__NO', 'Kiryat_Tivon__NOX', 'Kiryat_Tivon__SO2', 'Kiryat_Haim_Regavim__SO2', 'Kiryat_Haim_Regavim__NO', 'Kiryat_Haim_Regavim__NO2', 'Kiryat_Haim_Regavim__BENZN', 'Kiryat_Haim_Regavim__NOX', 'Kiryat_Haim_Regavim__O_Xyle', 'Kiryat_Haim_Regavim__TOLUEN', 'Kiryat_Haim_Regavim__EthylB', 'Kiryat_Binyamin__EthylB', 'Kiryat_Binyamin__O_Xyle', 'Kiryat_Binyamin__NOX', 'Kiryat_Binyamin__TOL', 'Kiryat_Binyamin__BENZN', 'Kiryat_Binyamin__NO', 'Kiryat_Binyamin__NO2', 'Kiryat_Binyamin__SO2', 'Kiryat_Ata__NO2', 'Kiryat_Ata__O3', 'Kiryat_Ata__NOX', 'Kiryat_Ata__SO2', 'Kiryat_Ata__NO', 'Park_Carmel__O3', 'Park_Carmel__NOx', 'Park_Carmel__NO', 'Park_Carmel__SO2', 'Atzmaut__NO2', 'Atzmaut__CO', 'Atzmaut__NOX', 'Atzmaut__NO', 'Nesher__NOX', 'Nesher__

In [ ]:
#Run a smoothing filter on the air quality data
from scipy import signal
b, a = signal.butter(14, 0.07)

if (0):
  smoothed_filtered_air_quality = filtered_air_quality
  smoothed_filtered_air_quality = smoothed_filtered_air_quality.fillna(-1)

  for column in smoothed_filtered_air_quality:
    if (column != "PollutionDate") and (column != "Date-Time") and (column != "DateTime"): 
      smoothed_filtered_air_quality[column] = signal.filtfilt(b, a, smoothed_filtered_air_quality[column], padlen=14)
      #smoothed_filtered_air_quality[column] = smoothed_filtered_air_quality[column].rolling(window=14).mean()

  filtered_air_quality = smoothed_filtered_air_quality

In [ ]:
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from scipy.stats.stats import pearsonr   
from patsy import dmatrices, dmatrix

sep = ' + '

#lag = 0
hospital = 'Rambam'

weather_cols = [col for col in filtered_air_quality.columns if 'Date' not in col]

formula = sep.join(["all_visits ~ 1 ", sep.join(weather_cols)])
#print(formula)

correlations = pd.DataFrame(columns=['Lag', 'Cor'])

for lag in range(-60, 60):
  filtered_air_quality["ShiftedPollutionDate"] = pd.DatetimeIndex(filtered_air_quality["PollutionDate"]) + pd.DateOffset(lag);
  merged_all = pd.merge(merged_data, filtered_air_quality, how='left', left_on='Date', right_on='ShiftedPollutionDate')
  #merged_all = merged_all.fillna(-1)

  #Split to train and test
  train_row_mask = (merged_all["hospital"] == hospital) & (merged_all['Date'] <= '2018-12-31')
  test_row_mask  = (merged_all["hospital"] == hospital) & (merged_all['Date'] > '2018-12-31')

  y, X = dmatrices(formula, merged_all)
  y_train = y[train_row_mask]
  X_train = X[train_row_mask]
  y_test = y[test_row_mask]
  y_test = [ i[0] for i in y_test]
  X_test = X[test_row_mask]

  #rf = RandomForestRegressor(n_estimators = 10)
  #rf.fit(X_train, y_train);
  #pred = rf.predict(X_test)

  model = sm.OLS(y_train.astype(float), X_train.astype(float)).fit()
  pred  = model.predict(X_test.astype(float))

  #curCorr = y_test.corr(pred)

  curCorr = pearsonr(pred, y_test)
  curCorr = curCorr[0]

  correlations = correlations.append({'Lag': lag, 'Cor': curCorr}, ignore_index=True)
  print(".", end='')


In [ ]:
from scipy import signal
b, a = signal.butter(5, 0.2)

correlations2 = correlations
y = signal.filtfilt(b, a, correlations2['Cor'], padlen=5)
correlations2['Cor'] = y
#print(correlations2['Cor'])

correlations2.plot(kind='scatter',x='Lag',y='Cor',color='red')

In [278]:
#Read weather and inversion data
weather = pd.read_csv("data/weather.csv")

weather.rename(columns=lambda x: re.sub('[- ()+]','_', x), inplace=True)

weather["date"] = pd.to_datetime(weather["date"],errors='coerce')


In [ ]:
# join all independent variables into one table
all_indep = air_quality.merge(weather, how='inner', left_on= "DateTime", right_on="date")
print(air_quality[['DateTime','Date_Time', 'PollutionDate']].head())
print(weather.head())

In [ ]:
import statsmodels.api as sm
import numpy as np
from patsy import dmatrices, dmatrix

sep = ' + '

#lag = 0
hospital = 'Rambam'

cols = [col for col in inversion.columns if 'date' not in col]

formula = sep.join(["all_visits ~ 1", sep.join(cols)])
print(formula)

correlations = pd.DataFrame(columns=['Lag', 'Cor'])

for lag in range(-60, 60):
#for lag in range(-6, 6):
  inversion["ShiftedPollutionDate"] = pd.DatetimeIndex(inversion["date"]) + pd.DateOffset(lag)

  merged_all = pd.merge(merged_data, inversion, how='left', left_on='Date', right_on='ShiftedPollutionDate')
  merged_all = merged_all.fillna(-1)

  #Split to train and test
  train_row_mask = (merged_all["hospital"] == hospital) & (merged_all['Date'] <= '2018-12-31')
  test_row_mask  = (merged_all["hospital"] == hospital) & (merged_all['Date'] > '2018-12-31')

  y, X = dmatrices(formula, merged_all)
  y_train = y[train_row_mask]
  X_train = X[train_row_mask]
  y_test = y[test_row_mask]
  y_test = [ i[0] for i in y_test]
  X_test = X[test_row_mask]

  model = sm.OLS(y_train.astype(float), X_train.astype(float)).fit()
  pred  = model.predict(X_test.astype(float))

  correlations = correlations.append({'Lag': lag, 'Cor': np.corrcoef(y_test, pred)[0,1]}, ignore_index=True)
  print(".", end='')

In [ ]:
from scipy import signal
b, a = signal.butter(5, 0.2)

correlations2 = correlations
y = signal.filtfilt(b, a, correlations2['Cor'], padlen=5)
correlations2['Cor'] = y
#print(correlations2['Cor'])

#correlations2['Cor'] = correlations2['Cor'].filtfilt(window=5).mean()

correlations2.plot(kind='line',x='Lag',y='Cor',color='red')

In [ ]:
from itertools import chain, combinations, product

inversion_basic = ['inversion_bywind', 'inversion_bytemp_delta', 'inversion_byboth']
climate = ["hmd_rlt_Bazan", "prs_lvl_hgt_Bazan", "tmp_air_dry_Bazan", "tmp_air_wet_Bazan", "tmp_dew_pnt_Bazan", "wind_dir_Bazan", "wind_spd_Bazan"]


xs = ['gaseous', 'particulate', 'climate', 'inversion']


all_subsets = chain.from_iterable(combinations(xs,n) for n in range(1, len(xs)+1))

print(list(all_subsets))

xp = ['climate', 'inversion']

inversion_cols = inversion_basic.copy()
inversion_cols.extend(map(lambda x: ':'.join(x), product(inversion_basic, climate)))
print(inversion_cols)

#factor_map = { 'gaseous' : aircols_gaseous, 'particulate' : aircols_particulate, 'climate' : climate_cols, 'inversion' : inversion_cols }